In [1]:
import matplotlib.pyplot as plt
from IPython.display import display, Markdown, Latex
import numpy as np
import json
import pandas as pd
from scipy import stats
import os

In [2]:
experiments = [
            {'id': '1', 'source':'imdb', 'target':'uwcse', 'predicate':'workedunder', 'to_predicate':'advisedby', 'arity': 2},
            #{'id': '2', 'source':'uwcse', 'target':'imdb', 'predicate':'advisedby', 'to_predicate':'workedunder', 'arity': 2},
            {'id': '3', 'source':'imdb', 'target':'cora', 'predicate':'workedunder', 'to_predicate':'samevenue', 'arity': 2},
            {'id': '4', 'source':'cora', 'target':'imdb', 'predicate':'samevenue', 'to_predicate':'workedunder', 'arity': 2},
            ##{'id': '5', 'source':'uwcse', 'target':'cora', 'predicate':'advisedby', 'to_predicate':'samevenue', 'arity': 2},
            ##{'id': '6', 'source':'cora', 'target':'uwcse', 'predicate':'samevenue', 'to_predicate':'advisedby', 'arity': 2},
            {'id': '7', 'source':'yeast', 'target':'twitter', 'predicate':'proteinclass', 'to_predicate':'accounttype', 'arity': 2},
            {'id': '8', 'source':'twitter', 'target':'yeast', 'predicate':'accounttype', 'to_predicate':'proteinclass', 'arity': 2},
            {'id': '9', 'source':'nell_sports', 'target':'nell_finances', 'predicate':'teamplayssport', 'to_predicate':'companyeconomicsector', 'arity': 2},
            #{'id': '10', 'source':'nell_finances', 'target':'nell_sports', 'predicate':'companyeconomicsector', 'to_predicate':'teamplayssport', 'arity': 2}
]

In [3]:
def load_data(experiments, metric, treeboostler=False):
    data = {}
    for item in experiments:
        experiment_title = item['id'] + '_' + item['source'] + '_' + item['target']
        
        if(treeboostler):
            file_path = os.path.split(os.getcwd())[0] + '/TreeBoostler/experiments-transfer/' + experiment_title + '/' + experiment_title + '.json'
        else:
            file_path = os.getcwd() + '/experiments-transfer-k-1/' + experiment_title + '/' + experiment_title + '_fasttext_{}.json'.format(metric)
        
        #if(os.path.isfile(file_path)):
        with open(file_path, 'r') as fp:
            results = json.load(fp)

            data[experiment_title] = results
    return data

def load_data_no_rev(experiments, metric, treeboostler=False):
    data = {}
    for item in experiments:
        experiment_title = item['id'] + '_' + item['source'] + '_' + item['target']
        
        if(treeboostler):
            file_path = os.path.split(os.getcwd())[0] + '/TreeBoostler/experiments-transfer/' + experiment_title + '/' + experiment_title + '.json'
        else:
            file_path = os.getcwd() + '/experiments-no-revision/' + experiment_title + '/' + experiment_title + '_fasttext_{}.json'.format(metric)
        
        #if(os.path.isfile(file_path)):
        with open(file_path, 'r') as fp:
            results = json.load(fp)

            data[experiment_title] = results
    return data

In [4]:
data_softcosine = load_data(experiments, 'softcosine')
data_euclidean = load_data(experiments, 'euclidean')
data_wmd = load_data(experiments, 'wmd')

data_softcosine_no_rev = load_data_no_rev(experiments, 'softcosine')
data_euclidean_no_rev = load_data_no_rev(experiments, 'euclidean')
data_wmd_no_rev = load_data_no_rev(experiments, 'wmd')

treeboostler_data = load_data(experiments, '', True)

for j in range(len(experiments)):
    dataset = experiments[j]['id'] + '_' + experiments[j]['source'] + '_' + experiments[j]['target']
    #if dataset in data['results']: 
    display(Markdown('# Results for ' + dataset))
    table = []
    for metric in ['AUC ROC', 'AUC PR', 'CLL']: #, 'Learning and Revision time', 'Inference time']:
        display(Markdown('## ' + metric))
        
        trans_softcosine = np.zeros(len(data_softcosine[str(dataset)][0]))
        trans_softcosine_no_rev = np.zeros(len(data_softcosine_no_rev[str(dataset)][0]))
        
        trans_euclidean = np.zeros(len(data_euclidean[str(dataset)][0]))
        trans_euclidean_no_rev = np.zeros(len(data_euclidean_no_rev[str(dataset)][0]))
        
        trans_wmd = np.zeros(len(data_wmd[str(dataset)][0]))
        trans_wmd_no_rev = np.zeros(len(data_wmd_no_rev[str(dataset)][0]))
        
        tree_no_rev = np.zeros(len(treeboostler_data[str(dataset)][0]))
        tree = np.zeros(len(treeboostler_data[str(dataset)][0]))
        
        for m in range(len(treeboostler_data[str(dataset)][0])):
            trans_softcosine += np.array([item['transfer'][metric] for item in data_softcosine[str(dataset)][m]])
            trans_softcosine_no_rev += np.array([item['transfer'][metric] for item in data_softcosine[str(dataset)][m]])
            
            trans_euclidean += np.array([item['transfer'][metric] for item in data_euclidean[str(dataset)][m]])
            trans_euclidean_no_rev += np.array([item['transfer'][metric] for item in data_euclidean[str(dataset)][m]])
            
            trans_wmd += np.array([item['transfer'][metric] for item in data_wmd[str(dataset)][m]])
            trans_wmd_no_rev += np.array([item['transfer'][metric] for item in data_wmd[str(dataset)][m]])
            
            tree_no_rev += np.array([item['transfer']['parameter'][metric] for item in treeboostler_data[str(dataset)][m]])
            tree += np.array([item['transfer'][metric] for item in treeboostler_data[str(dataset)][m]])
            
        trans_softcosine /= len(data_softcosine[str(dataset)])
        trans_softcosine_no_rev /= len(data_softcosine_no_rev[str(dataset)])
        
        trans_euclidean /= len(data_euclidean[str(dataset)])
        trans_euclidean_no_rev /= len(data_euclidean_no_rev[str(dataset)])
        
        trans_wmd /= len(data_wmd[str(dataset)])
        trans_wmd_no_rev /= len(data_wmd_no_rev[str(dataset)])
        
        tree /= len(treeboostler_data[str(dataset)])
        tree_no_rev /= len(treeboostler_data[str(dataset)])
        
        pvalue_table = []
        tvalue, pvalue = stats.ttest_rel(trans_softcosine,tree)
        #pvalue2 = stats.ttest_rel(tr_sum,tree_no_rev)
        #print(pvalue)
        pvalue_table.append(['TransBoostler Soft Cosine', '%.3f' % (pvalue), '%.3f' % (tvalue)])
        
        tvalue, pvalue = stats.ttest_rel(trans_euclidean,tree)
        #pvalue2 = stats.ttest_rel(tr_sum,tree_no_rev)
        #print(pvalue)
        pvalue_table.append(['TransBoostler Euclidean', '%.3f' % (pvalue), '%.3f' % (tvalue)])
        
        tvalue, pvalue = stats.ttest_rel(trans_wmd,tree)
        #pvalue2 = stats.ttest_rel(tr_sum,tree_no_rev)
        #print(pvalue)
        pvalue_table.append(['TransBoostler WMD', '%.3f' % (pvalue), '%.3f' % (tvalue)])
        
        # No Revision
        
        tvalue, pvalue = stats.ttest_rel(trans_softcosine_no_rev,tree_no_rev)
        #pvalue2 = stats.ttest_rel(tr_sum,tree_no_rev)
        #print(pvalue)
        pvalue_table.append(['TransBoostler* Soft Cosine', '%.3f' % (pvalue), '%.3f' % (tvalue)])
        
        tvalue, pvalue = stats.ttest_rel(trans_euclidean_no_rev,tree_no_rev)
        #pvalue2 = stats.ttest_rel(tr_sum,tree_no_rev)
        #print(pvalue)
        pvalue_table.append(['TransBoostler* Euclidean', '%.3f' % (pvalue), '%.3f' % (tvalue)])
        
        tvalue, pvalue = stats.ttest_rel(trans_wmd_no_rev,tree_no_rev)
        #pvalue2 = stats.ttest_rel(tr_sum,tree_no_rev)
        #print(pvalue)
        pvalue_table.append(['TransBoostler* WMD', '%.3f' % (pvalue), '%.3f' % (tvalue)])
        
        display(pd.DataFrame(pvalue_table, columns=['TreeBoostler', 'p-value', 't-stat']))

        #for m in range(len(data['results'][str(dataset)])):
        #    print('Experiment %s' % (m+1))
        #    sc = [item['rdn_b'][metric] for item in data['results'][str(dataset)][m]]
        #    sc_rdn = [item['rdn'][metric] for item in data['results'][str(dataset)][m]]
        #    pr = [item['transfer']['parameter'][metric] for item in data['results'][str(dataset)][m]]
        #    tr = [item['transfer'][metric] for item in data['results'][str(dataset)][m]] 
        #    print('Scratch RDN-B: %s' % sc)
        #    print('Scratch RDN: %s' % sc_rdn)
        #    print('Transfer: %s' % pr)
        #    print('Transfer & Revision: %s' % tr) 
        #    print('\n')


# Results for 1_imdb_uwcse

## AUC ROC

,TreeBoostler,p-value,t-stat
0,TransBoostler Soft Cosine,0.542,-0.666
1,TransBoostler Euclidean,0.567,-0.622
2,TransBoostler WMD,0.526,-0.693
3,TransBoostler* Soft Cosine,0.052,2.745
4,TransBoostler* Euclidean,0.078,2.361
5,TransBoostler* WMD,0.125,1.935


## AUC PR

,TreeBoostler,p-value,t-stat
0,TransBoostler Soft Cosine,0.433,-0.871
1,TransBoostler Euclidean,0.238,-1.387
2,TransBoostler WMD,0.318,-1.140
3,TransBoostler* Soft Cosine,0.710,-0.400
4,TransBoostler* Euclidean,0.437,-0.863
5,TransBoostler* WMD,0.509,-0.724


...

,TreeBoostler,p-value,t-stat
0,TransBoostler Soft Cosine,0.002,-20.417
1,TransBoostler Euclidean,0.004,-15.643
2,TransBoostler WMD,0.003,-18.656
3,TransBoostler* Soft Cosine,0.003,-19.550
4,TransBoostler* Euclidean,0.004,-15.040
5,TransBoostler* WMD,0.003,-18.170


## AUC PR

,TreeBoostler,p-value,t-stat
0,TransBoostler Soft Cosine,0.405,1.046
1,TransBoostler Euclidean,0.847,-0.218
2,TransBoostler WMD,0.296,1.402
3,TransBoostler* Soft Cosine,0.280,1.467
4,TransBoostler* Euclidean,0.934,0.093
5,TransBoostler* WMD,0.161,2.177


## CLL

,TreeBoostler,p-value,t-stat
0,TransBoostler Soft Cosine,0.000,-87.171
1,TransBoostler Euclidean,0.001,-44.141
2,TransBoostler WMD,0.001,-40.703
3,TransBoostler* Soft Cosine,0.000,-55.536
4,TransBoostler* Euclidean,0.001,-32.663
5,TransBoostler* WMD,0.000,-58.961
